In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [36]:
path= "/content/drive/MyDrive/Bootcamp/BBDD/titanic_data.csv"
df=pd.read_csv(path)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  891 non-null    int64  
 1   survived    891 non-null    int64  
 2   sex         891 non-null    object 
 3   age         891 non-null    float64
 4   sibsp       891 non-null    int64  
 5   parch       891 non-null    int64  
 6   fare        891 non-null    float64
 7   embarked    891 non-null    object 
 8   class       891 non-null    object 
 9   who         891 non-null    object 
 10  alone       891 non-null    int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


In [38]:
df.isnull().sum()

,0
Unnamed: 0,0
survived,0
sex,0
age,0
sibsp,0
parch,0
fare,0
embarked,0
class,0
who,0


* Previamente se hizo limpieza de columnas y datos nulos.

In [39]:
df = df.drop('Unnamed: 0', axis=1) # Elimino la columna Unnamed

In [40]:
df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
survived,891.0,0.38,0.49,0.00,0.00,0.00,1.0,1.00
age,891.0,29.94,13.17,0.42,22.00,29.00,36.0,80.00
sibsp,891.0,0.52,1.10,0.00,0.00,0.00,1.0,8.00
parch,891.0,0.38,0.81,0.00,0.00,0.00,0.0,6.00
fare,891.0,32.20,49.69,0.00,7.91,14.45,31.0,512.33
alone,891.0,0.60,0.49,0.00,0.00,1.00,1.0,1.00


In [41]:
df.head()

,survived,sex,age,sibsp,parch,fare,embarked,class,who,alone
0,0,male,22.0,1,0,7.2500,S,Third,man,0
1,1,female,38.0,1,0,71.2833,C,First,woman,0
2,1,female,26.0,0,0,7.9250,S,Third,woman,1
3,1,female,35.0,1,0,53.1000,S,First,woman,0
4,0,male,35.0,0,0,8.0500,S,Third,man,1


In [42]:
# Separo mi Target y features.
X = df.drop(columns= 'survived')
y = df['survived']

In [43]:
# Split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
# Dividir features.
num_cols = ["age", "sibsp", "parch", "fare", "alone"]
ord_cols = ["class"]
nom_cols = ["sex", "embarked", "who"]

In [45]:
# Dividir features.
X_num_train = X_train[num_cols]
X_ord_train = X_train[ord_cols]
X_nom_train = X_train[nom_cols]

In [46]:
# Realizo escalado con StandardScaler.
scaler = StandardScaler()
X_num_train_scaled = scaler.fit_transform(X_num_train)
X_num_test = scaler.transform(X_test[num_cols])

In [47]:
# Codifico las variables categóricas ordinales.
ordinal = OrdinalEncoder(categories=[["Third", "Second", "First"]])
X_ord_train = ordinal.fit_transform(X_train[ord_cols])
X_ord_test = ordinal.transform(X_test[ord_cols])

In [48]:
# Codificamos variables categóricas nominales.
ohe = OneHotEncoder(sparse_output=False)
X_nom_train = ohe.fit_transform(X_train[nom_cols])
X_nom_test = ohe.transform(X_test[nom_cols])

In [49]:
# Junto todo
X_train_final = np.concatenate([X_num_train, X_ord_train, X_nom_train], axis=1)
X_test_final = np.concatenate([X_num_test, X_ord_test, X_nom_test], axis=1)

# CREACIÓN DE LOS 5 MODELOS
---

In [50]:
modelos = {
    'Regresión Logística': LogisticRegression(random_state=42, max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'Árbol de Decisión': DecisionTreeClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(random_state=42, verbose=-1)
}

In [51]:
# Validación cruzada
resultados_iniciales = {}
print("\nResultados iniciales (sin optimizar):")
print("-" * 40)

for nombre, modelo in modelos.items():
    # Validación cruzada = probar el modelo 5 veces con diferentes divisiones
    scores = cross_val_score(modelo, X_train_final, y_train, cv=5, scoring='accuracy')
    promedio = scores.mean()
    resultados_iniciales[nombre] = promedio

    print(f"{nombre}: {promedio:.3f} ± {scores.std():.3f}")


Resultados iniciales (sin optimizar):
----------------------------------------
Regresión Logística: 0.817 ± 0.023
KNN: 0.723 ± 0.032
Árbol de Decisión: 0.777 ± 0.028
XGBoost: 0.801 ± 0.018
LightGBM: 0.820 ± 0.011


# OPTIMIZAR LOS MODELOS
---

In [62]:
# Definir qué parámetros probar para cada modelo
parametros_a_probar = {
    'Regresión Logística': {
        'C': [0.01, 0.1, 1, 10, 100],
        'max_iter': [1000]},
    'KNN': {
        'n_neighbors': [5, 7, 9, 11, 15, 21],
        'weights': ['uniform', 'distance']},
    'Árbol de Decisión': {
        'max_depth': [3, 5, 7, 10, None],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 5]
    },
    'XGBoost': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 4, 5, 6],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample':[0.8, 10]
    },
    'LightGBM': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 4, 5, 6],
        'learning_rate': [0.01, 0.1, 0.2],
        'num_leaves': [20, 31, 50]}}

In [63]:
# Optimizar cada modelo
modelos_optimizados = {}
mejores_parametros = {}

print("\nOptimizando modelos...")
print("-" * 30)

for nombre, modelo in modelos.items():
    print(f"Optimizando {nombre}...")

    # GridSearchCV prueba todas las combinaciones de parámetros
    grid_search = GridSearchCV(
        modelo,
        parametros_a_probar[nombre],
        cv=5,  # 5 validaciones cruzadas (más robusto que 3)
        scoring='accuracy',
        n_jobs=-1  # Usar todos los cores disponibles
    )

    grid_search.fit(X_train_final, y_train)

    # Guardar el mejor modelo y sus parámetros
    modelos_optimizados[nombre] = grid_search.best_estimator_
    mejores_parametros[nombre] = grid_search.best_params_

    print(f"  Mejor score: {grid_search.best_score_:.3f}")
    print(f"  Mejores parámetros: {grid_search.best_params_}")


Optimizando modelos...
------------------------------
Optimizando Regresión Logística...
  Mejor score: 0.819
  Mejores parámetros: {'C': 10, 'max_iter': 1000}
Optimizando KNN...
  Mejor score: 0.732
  Mejores parámetros: {'n_neighbors': 7, 'weights': 'distance'}
Optimizando Árbol de Decisión...
  Mejor score: 0.827
  Mejores parámetros: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
Optimizando XGBoost...
  Mejor score: 0.833
  Mejores parámetros: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Optimizando LightGBM...
  Mejor score: 0.841
  Mejores parámetros: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'num_leaves': 20}


# EVALUACIÓN MODELOS OPTIMIZADOS
---

In [64]:
resultados_finales = {}
predicciones = {}

print("\nResultados finales (optimizados):")
print("-" * 40)

for nombre, modelo in modelos_optimizados.items():
    # Entrenar con todos los datos de entrenamiento
    modelo.fit(X_train_final, y_train)

    # Hacer predicciones en datos de prueba
    y_pred = modelo.predict(X_test_final)

    # Calcular accuracy
    accuracy = accuracy_score(y_test, y_pred)
    resultados_finales[nombre] = accuracy
    predicciones[nombre] = y_pred

    print(f"{nombre}: {accuracy:.3f}")


Resultados finales (optimizados):
----------------------------------------
Regresión Logística: 0.765
KNN: 0.413
Árbol de Decisión: 0.788
XGBoost: 0.771
LightGBM: 0.782


In [65]:
# Crear tabla de comparación
comparacion = pd.DataFrame({
    'Modelo': list(resultados_iniciales.keys()),
    'Accuracy Inicial': [resultados_iniciales[modelo] for modelo in resultados_iniciales.keys()],
    'Accuracy Optimizado': [resultados_finales[modelo] for modelo in resultados_finales.keys()],
})

comparacion['Mejora'] = comparacion['Accuracy Optimizado'] - comparacion['Accuracy Inicial']
comparacion = comparacion.sort_values('Accuracy Optimizado', ascending=False)

print("\nTabla de comparación:")
print(comparacion.round(3))


Tabla de comparación:
                Modelo  Accuracy Inicial  Accuracy Optimizado  Mejora
2    Árbol de Decisión             0.777                0.788   0.011
4             LightGBM             0.820                0.782  -0.038
3              XGBoost             0.801                0.771  -0.030
0  Regresión Logística             0.817                0.765  -0.052
1                  KNN             0.723                0.413  -0.310


In [66]:
# Encontrar el mejor modelo
mejor_modelo = comparacion.iloc[0]['Modelo']
print("\n🔍 ANÁLISIS DE LOS RESULTADOS:")
print("-" * 50)


🔍 ANÁLISIS DE LOS RESULTADOS:
--------------------------------------------------


In [67]:
# Analizar qué pasó con cada modelo
print("¿Por qué algunos modelos empeoraron?")
print("• KNN: Probablemente overfitting con pocos vecinos")
print("• LightGBM/XGBoost: Puede que los parámetros por defecto fueran mejores")
print("• Regresión Logística: Regularización muy fuerte o muy débil")
print("• Árbol de Decisión: ¡Mejoró! La poda ayudó a evitar overfitting")

print(f"\n🏆 GANADOR: {mejor_modelo}")
print(f"Accuracy: {comparacion.iloc[0]['Accuracy Optimizado']:.3f}")
print(f"Parámetros: {mejores_parametros[mejor_modelo]}")

¿Por qué algunos modelos empeoraron?
• KNN: Probablemente overfitting con pocos vecinos
• LightGBM/XGBoost: Puede que los parámetros por defecto fueran mejores
• Regresión Logística: Regularización muy fuerte o muy débil
• Árbol de Decisión: ¡Mejoró! La poda ayudó a evitar overfitting

🏆 GANADOR: Árbol de Decisión
Accuracy: 0.788
Parámetros: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
